In [1]:
import pandas as pd

In [2]:
df1 = pd.read_excel("BD_IEET_2015.xlsx")
df2 = pd.read_csv("Más especies de BD_IEET.csv")
df_taxonomia = pd.read_excel('../../Datos EIDOS/EIDOS_taxonomia.xlsx')

In [5]:
df = pd.concat([df1, df2])
df = df.fillna("")

In [ ]:
# Quitar parentesis y palabras que empiezan por mayuscula y numeros
df_taxonomia['lowered_name'] = df_taxonomia['name'].str.replace(r'[()&]', '', regex=True).apply(
    lambda x: x[0].lower() + ' '.join([w for w in x[1:].split() if w == w.lower() and not w.isdigit()])
    )

def find_best_match(species):
    row_words = species.replace('(', '').replace(')', '').lower().split()
    if len(row_words) < 2:
        return None # No hay suficientes palabras para buscar
    # Filtrar candidatos
    candidates = df_taxonomia[df_taxonomia['lowered_name'].str.contains(' '.join(row_words[:2]))]
    # Si hay una categoría taxonómica, refinar los candidatos
    if len(row_words) > 3 and row_words[2] in ['f.', 'subsp.', 'var.']:
        candidates = candidates[candidates['lowered_name'].str.contains(f' {row_words[3]} ')]
    else:
        candidates = candidates[~candidates['lowered_name'].str.contains(r'\b(?: f\. | subsp\. | var\. )\b', regex=True)]
    if candidates.empty:
        return None
    # Calcular las puntuaciones de coincidencia
    row_words_set = set(row_words)
    candidates['score'] = candidates['lowered_name'].apply(lambda c: 
        0.9 * len(set(c.split()) & row_words_set) / len(row_words) +
        0.1 * len(set(c.split()) & row_words_set) / len(c.split())
    )
    # Obtener el mejor candidato
    best_match = candidates.loc[candidates['score'].idxmax()]
    return best_match['taxonid']

species_id = {species: find_best_match(species) for species in set(df["Nombre"].to_list())}
df['idtaxon'] = df["Nombre"].apply(lambda x: species_id[x])
df.to_excel('IEET_ubicaciones_idtaxon.xlsx', index=False)